In [1]:
import itertools
import logging
import pathlib
import sys
from typing import Optional

import numpy as np
import pandas as pd
import toml
from copairs.map import run_pipeline
from pycytominer import feature_select

# imports src
sys.path.append("../")
from src import utils

# setting up logger
logging.basicConfig(
    filename="map_analysis_testing.log",
    level=logging.DEBUG,
    format="%(levelname)s:%(asctime)s:%(name)s:%(message)s",
)

## Helper functions
Set of helper functions to help out throughout the notebook

In [2]:
## Helper function


def shuffle_meta_labels(
    dataset: pd.DataFrame, target_col: str, seed: Optional[int] = 0
) -> pd.DataFrame:
    """shuffles labels or values within a single selected column

    Parameters
    ----------
    dataset : pd.DataFrame
        dataframe containing the dataset

    target_col : str
        Column to select in order to conduct the shuffling

    seed : int
        setting random seed

    Returns
    -------
    pd.DataFrame
        shuffled dataset

    Raises
    ------
    TypeError
        raised if incorrect types are provided
    """
    # setting seed
    np.random.seed(seed)

    # type checking
    if not isinstance(target_col, str):
        raise TypeError("'target_col' must be a string type")
    if not isinstance(dataset, pd.DataFrame):
        raise TypeError("'dataset' must be a pandas dataframe")

    # selecting column, shuffle values within column, add to dataframe
    dataset[target_col] = np.random.permutation(dataset[target_col].values)
    return dataset


def shuffle_features(feature_vals: np.array, seed: Optional[int] = 0) -> np.array:
    """suffles all values within feature space

    Parameters
    ----------
    feature_vals : np.array
        shuffled

    seed : Optional[int]
        setting random seed

    Returns
    -------
    np.array
        Returns shuffled values within the feature space

    Raises
    ------
    TypeError
        Raised if a numpy array is not provided
    """
    # setting seed
    np.random.seed(seed)

    # shuffle given array
    if not isinstance(feature_vals, np.ndarray):
        raise TypeError("'feature_vals' must be a numpy array")
    if feature_vals.ndim != 2:
        raise TypeError("'feature_vals' must be a 2x2 matrix")

    # creating a copy for feature vales to prevent overwriting of global variables
    feature_vals = np.copy(feature_vals)

    # shuffling feature space
    n_cols = feature_vals.shape[1]
    for col_idx in range(0, n_cols):
        # selecting column, shuffle, and update:
        feature_vals[:, col_idx] = np.random.permutation(feature_vals[:, col_idx])

    return feature_vals

## Setting up Paths and loading data

In [3]:
# load in the treatment groups
ground_truth = pathlib.Path(
    "../../4.sc_Morphology_Neural_Network_MLP_Model/MLP_utils/ground_truth.toml"
).resolve(strict=True)
# load in the ground truth
ground_truth = toml.load(ground_truth)
apoptosis_ground_truth = ground_truth["Apoptosis"]["apoptosis_groups_list"]
pyroptosis_ground_truth = ground_truth["Pyroptosis"]["pyroptosis_groups_list"]
control_ground_truth = ground_truth["Healthy"]["healthy_groups_list"]

In [4]:
single_cell_data = pathlib.Path(
    f"../../2.Nomic_nELISA_Analysis/Data/clean/Plate2/nELISA_plate_430420_PBMC_clean.parquet"
).resolve(strict=True)
df = pd.read_parquet(single_cell_data)

In [5]:
df

,plate_name,plate_barcode,position_x,cell_type,incubation inducer,inhibitor,inhibitor_concentration_value,inhibitor_concentration_unit,inhibitor_concentration,inducer1,...,VEGF-D [NSU],VEGFR-1 [NSU],WISP-1 (CCN4) [NSU],XCL1 (Lymphotactin) [NSU],Treatment,Dose,oneb_Treatment_Dose_Inhibitor_Dose,twob_Treatment_Dose_Inhibitor_Dose,threeb_Treatment_Dose_Inhibitor_Dose,fourb_Treatment_Dose_Inhibitor_Dose
0,70117_20230210MM1_P1,430420,B06,PBMC,6_h,DMSO,0.025,%,0.030,DMSO,...,0.258834,0.238358,0.524276,0.250670,DMSO,0.100_%,DMSO_0.100_%_DMSO_0.025_%,DMSO_DMSO_0.100_%,DMSO__0.100_%__DMSO_0.030,DMSO__0.100_%__DMSO__0.030
1,70117_20230210MM1_P1,430420,C06,PBMC,6_h,DMSO,0.025,%,0.030,DMSO,...,0.381170,0.168645,0.455092,0.228752,DMSO,0.100_%,DMSO_0.100_%_DMSO_0.025_%,DMSO_DMSO_0.100_%,DMSO__0.100_%__DMSO_0.030,DMSO__0.100_%__DMSO__0.030
2,70117_20230210MM1_P1,430420,I06,PBMC,6_h,DMSO,0.025,%,0.030,DMSO,...,0.182956,0.263281,0.213596,0.064645,DMSO,0.100_%,DMSO_0.100_%_DMSO_0.025_%,DMSO_DMSO_0.100_%,DMSO__0.100_%__DMSO_0.030,DMSO__0.100_%__DMSO__0.030
3,70117_20230210MM1_P1,430420,J06,PBMC,6_h,DMSO,0.025,%,0.030,DMSO,...,0.582053,0.087565,0.140992,0.234191,DMSO,0.100_%,DMSO_0.100_%_DMSO_0.025_%,DMSO_DMSO_0.100_%,DMSO__0.100_%__DMSO_0.030,DMSO__0.100_%__DMSO__0.030
4,70117_20230210MM1_P1,430420,B07,PBMC,6_h,DMSO,0.025,%,0.030,DMSO,...,0.264141,0.296782,0.541689,0.167078,DMSO,0.100_%,DMSO_0.100_%_DMSO_0.025_%,DMSO_DMSO_0.100_%,DMSO__0.100_%__DMSO_0.030,DMSO__0.100_%__DMSO__0.030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,70117_20230210MM1_P1,430420,O05,PBMC,6_h,Media_ctr,NaN,nan,0.0,media_ctr,...,0.275941,0.312566,0.286031,0.288358,media_ctr,0.0_nan,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0.0_nan,media_ctr__0.0_nan__Media_ctr_0.0,media_ctr__0.0_nan__Media_ctr__0.0
150,70117_20230210MM1_P1,430420,O10,PBMC,6_h,Media_ctr,NaN,nan,0.0,media_ctr,...,0.450715,0.178011,0.621119,0.229238,media_ctr,0.0_nan,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0.0_nan,media_ctr__0.0_nan__Media_ctr_0.0,media_ctr__0.0_nan__Media_ctr__0.0
151,70117_20230210MM1_P1,430420,O11,PBMC,6_h,Media,NaN,nan,0.0,media_ctr,...,0.308965,0.268730,0.613026,0.254080,media_ctr,0.0_nan,media_ctr_0.0_0_Media_0.0_0,media_ctr_Media_0.0_nan,media_ctr__0.0_nan__Media_0.0,media_ctr__0.0_nan__Media__0.0
152,70117_20230210MM1_P1,430420,B12,PBMC,6_h,Media_ctr,NaN,nan,0.0,media_ctr,...,0.161322,0.296984,0.625991,0.297158,media_ctr,0.0_nan,media_ctr_0.0_0_Media_ctr_0.0_0,media_ctr_Media_ctr_0.0_nan,media_ctr__0.0_nan__Media_ctr_0.0,media_ctr__0.0_nan__Media_ctr__0.0


In [6]:
# rename columns
df = df.rename(
    columns={
        "position_x": "Metadata_Well",
        "oneb_Treatment_Dose_Inhibitor_Dose": "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
    }
)

In [7]:
# out paths
map_out_dir = pathlib.Path("../data/processed/mAP_scores/secretome/")
map_out_dir.mkdir(exist_ok=True, parents=True)

# regular data output
# saving to csv
regular_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_regular_treatment.csv")

# shuffled data output
shuffled_feat_map_path = pathlib.Path(map_out_dir / "mAP_scores_shuffled_treatment.csv")

# shuffled feature space output
shuffled_feat_space_map_path = pathlib.Path(
    map_out_dir / "mAP_scores_shuffled_feature_space_treatment.csv"
)

### Clean up data

In [8]:
# keep columns that contain Metdata and ['NSU']
df = df.filter(regex="Metadata|NSU")
df.head()

df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()
# replace values in the oneb_Metadata_Treatment_Dose_Inhibitor_Dose column
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_0.100_ug_per_ml_DMSO_0.000_%", "Flagellin_0.100_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_0.100_ug_per_ml_DMSO_0.0_%", "Flagellin_0.100_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_1.000_ug_per_ml_DMSO_0.0_%", "Flagellin_1.000_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("Flagellin_1.000_0_DMSO_0.025_%", "Flagellin_1.000_ug_per_ml_DMSO_0.025_%")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_1.000_ug_per_ml_DMSO_0.000_%", "Flagellin_1.000_ug_per_ml_DMSO_0.025_%"
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("media_ctr_0.0_0_Media_0_0", "Media")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("media_ctr_0.0_0_Media_ctr_0.0_0", "Media")
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace(
    "Flagellin_1.000_0_Disulfiram_1.000_uM",
    "Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM",
)
df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"] = df[
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"
].replace("media_ctr_0.0_0_Media_0.0_0", "Media")
print(len(df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique()))

37


In [9]:
# add apoptosis, pyroptosis and healthy columns to dataframe
df["Apoptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in apoptosis_ground_truth,
    axis=1,
)
df["Pyroptosis"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in pyroptosis_ground_truth,
    axis=1,
)
df["Control"] = df.apply(
    lambda row: row["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
    in control_ground_truth,
    axis=1,
)

# merge apoptosis, pyroptosis, and healthy columns into one column
df["Metadata_labels"] = df.apply(
    lambda row: "Apoptosis"
    if row["Apoptosis"]
    else "Pyroptosis"
    if row["Pyroptosis"]
    else "Control",
    axis=1,
)
# # drop apoptosis, pyroptosis, and healthy columns
df.drop(columns=["Apoptosis", "Pyroptosis", "Control"], inplace=True)

In [10]:
# output directories
map_out_dir = pathlib.Path("../data/processed/mAP_scores/")
map_out_dir.mkdir(parents=True, exist_ok=True)

### mAP Pipeline Parameters

The null size needs to be determined for the mAP pipeline. This is the size of the null class that is used to determine the mAP score.

In [11]:
tmp = (
    df.groupby(["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"])
    .count()
    .reset_index()[["Metadata_Well", "oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]]
)
# get the Pyroptosis number of Metadata_Well
# get the counts of each oneb_Metadata_Treatment_Dose_Inhibitor_Dose
min_count = tmp["Metadata_Well"].min()
print(min_count)
tmp

4


,Metadata_Well,oneb_Metadata_Treatment_Dose_Inhibitor_Dose
0,8,DMSO_0.100_%_DMSO_0.025_%
1,4,DMSO_0.100_%_DMSO_1.000_%
2,4,DMSO_0.100_%_Z-VAD-FMK_100.000_uM
3,4,DMSO_0.100_%_Z-VAD-FMK_30.000_uM
4,4,Disulfiram_0.100_uM_DMSO_0.025_%
5,4,Disulfiram_1.000_uM_DMSO_0.025_%
6,4,Disulfiram_2.500_uM_DMSO_0.025_%
7,4,Flagellin_0.100_ug_per_ml_DMSO_0.025_%
8,4,Flagellin_1.000_ug_per_ml_DMSO_0.025_%
9,4,Flagellin_1.000_ug_per_ml_Disulfiram_1.000_uM


In [12]:
# generate the permutations of cell death labels via itertools
pos_samby_permutations = list(itertools.combinations(df["Metadata_labels"].unique(), 2))
pos_samby_permutations

[('Control', 'Pyroptosis'),
 ('Control', 'Apoptosis'),
 ('Pyroptosis', 'Apoptosis')]

In [13]:
pos_sameby = ["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"]
pos_diffby = ["Metadata_Well"]

neg_sameby = ["Metadata_labels"]
neg_diffby = [
    "oneb_Metadata_Treatment_Dose_Inhibitor_Dose",
]

null_size = min_count
batch_size = 1

# number of resampling
n_resamples = 10

### mAP analysis for non-shuffled data

In [14]:
results_df = pd.DataFrame(
    columns=[
        "Metadata_Well",
        "Metadata_labels",
        "average_precision",
        "p_value",
        "n_pos_pairs",
        "n_total_pairs",
        "shuffled",
        "comparison",
    ]
)

In [15]:
for i in df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique():

    # manually get treatment
    tmp = df[
        df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].str.contains(i)
    ].reset_index(drop=True)
    # get the label
    label = tmp["Metadata_labels"].unique().tolist()[0]
    # add all labels to the df except for the LPS treatment label
    tmp1 = df[~df["Metadata_labels"].str.contains(label)].reset_index(drop=True)
    # concat tmp and tmp1
    tmp1 = pd.concat([tmp, tmp1]).reset_index(drop=True)

    # drop rows that contain the label
    _pos_samby_permutations = [x for x in pos_samby_permutations if label in x]

    for j in _pos_samby_permutations:
        tmp1 = tmp1[tmp1["Metadata_labels"].isin(j)].reset_index(drop=True)

        # spliting metadata and raw feature values
        logging.info("splitting data set into metadata and raw feature values")
        df_meta, df_feats = utils.split_data(tmp1)
        df_feats = np.array(df_feats)
        try:
            # execute pipeline on negative control with trianing dataset with cp features

            logging.info(f"Running pipeline on CP features using phenotype")
            result = run_pipeline(
                meta=df_meta,
                feats=df_feats,
                pos_sameby=pos_sameby,
                pos_diffby=pos_diffby,
                neg_sameby=neg_sameby,
                neg_diffby=neg_diffby,
                batch_size=batch_size,
                null_size=null_size,
            )

            result["shuffled"] = "non-shuffled"
            comparison = i
            comparison = comparison + "_" + "_vs_".join(j)

            result["comparison"] = comparison

        except ZeroDivisionError as e:
            logging.warning(f"{e} captured on phenotye:. Skipping")

        # concatenating all datasets
        results_df = pd.concat([results_df, result], ignore_index=True)
results_df.to_csv(regular_feat_map_path, index=False)
results_df.head()

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/2176 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_532780/178303504.py:51: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result], ignore_index=True)


  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/117 [00:00<?, ?it/s]

  0%|          | 0/2176 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled,comparison,oneb_Metadata_Treatment_Dose_Inhibitor_Dose
0,B06,Control,1.0,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
1,C06,Control,1.0,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
2,I06,Control,1.0,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
3,J06,Control,1.0,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
4,B07,Control,1.0,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%


### mAP analysis for shuffled data (Phenotype)

In [16]:
results_df = pd.DataFrame(
    columns=[
        "Metadata_Well",
        "Metadata_labels",
        "average_precision",
        "p_value",
        "n_pos_pairs",
        "n_total_pairs",
        "shuffled",
        "comparison",
    ]
)

In [17]:
for i in df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].unique():

    # manually get treatment
    tmp = df[
        df["oneb_Metadata_Treatment_Dose_Inhibitor_Dose"].str.contains(i)
    ].reset_index(drop=True)
    # get the label
    label = tmp["Metadata_labels"].unique().tolist()[0]
    # add all labels to the df except for the LPS treatment label
    tmp1 = df[~df["Metadata_labels"].str.contains(label)].reset_index(drop=True)
    # concat tmp and tmp1
    tmp1 = pd.concat([tmp, tmp1]).reset_index(drop=True)

    # drop rows that contain the label
    _pos_samby_permutations = [x for x in pos_samby_permutations if label in x]

    for j in _pos_samby_permutations:
        tmp1 = tmp1[tmp1["Metadata_labels"].isin(j)].reset_index(drop=True)

        # spliting metadata and raw feature values
        logging.info("splitting data set into metadata and raw feature values")
        df_meta, df_feats = utils.split_data(tmp1)
        df_feats = np.array(df_feats)
        seed = np.random.randint(0, 100)

        # shuffling the features, this will overwrite the generated feature space from above with the shuffled one
        df_feats = shuffle_features(feature_vals=df_feats, seed=seed)

        try:
            # execute pipeline on negative control with trianing dataset with cp features

            logging.info(f"Running pipeline on CP features using phenotype")
            result = run_pipeline(
                meta=df_meta,
                feats=df_feats,
                pos_sameby=pos_sameby,
                pos_diffby=pos_diffby,
                neg_sameby=neg_sameby,
                neg_diffby=neg_diffby,
                batch_size=batch_size,
                null_size=null_size,
            )

            result["shuffled"] = "shuffled"
            comparison = i
            comparison = comparison + "_" + "_vs_".join(j)

            result["comparison"] = comparison

        except ZeroDivisionError as e:
            logging.warning(f"{e} captured on phenotye:. Skipping")

        # concatenating all datasets
        results_df = pd.concat([results_df, result], ignore_index=True)

# saving to csv
results_df.to_csv(shuffled_feat_space_map_path, index=False)
results_df

  0%|          | 0/130 [00:00<?, ?it/s]

  0%|          | 0/2176 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/tmp/ipykernel_532780/1162644311.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, result], ignore_index=True)


  0%|          | 0/28 [00:00<?, ?it/s]

...

  0%|          | 0/117 [00:00<?, ?it/s]

  0%|          | 0/2176 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

,Metadata_Well,Metadata_labels,average_precision,p_value,n_pos_pairs,n_total_pairs,shuffled,comparison,oneb_Metadata_Treatment_Dose_Inhibitor_Dose
0,B06,Control,1.000000,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
1,C06,Control,1.000000,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
2,I06,Control,1.000000,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
3,J06,Control,1.000000,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
4,B07,Control,1.000000,1.0,7.0,7.0,non-shuffled,DMSO_0.100_%_DMSO_0.025_%_Control_vs_Pyroptosis,DMSO_0.100_%_DMSO_0.025_%
...,...,...,...,...,...,...,...,...,...
5715,C11,Pyroptosis,0.043723,0.6,3.0,67.0,non-shuffled,Media_Control_vs_Pyroptosis,LPS_Nigericin_100.000_ug_per_ml_3.000_uM_DMSO_...
5716,J02,Pyroptosis,0.055159,0.4,3.0,67.0,non-shuffled,Media_Control_vs_Pyroptosis,LPS_100.000_ug_per_ml_DMSO_0.025_%
5717,J03,Pyroptosis,0.059474,0.4,3.0,67.0,non-shuffled,Media_Control_vs_Pyroptosis,LPS_100.000_ug_per_ml_DMSO_0.025_%
5718,J08,Pyroptosis,0.091635,0.4,3.0,67.0,non-shuffled,Media_Control_vs_Pyroptosis,LPS_100.000_ug_per_ml_DMSO_0.025_%
